# Some Models
> To avoid copy-pasta #2

In [ ]:
# default_exp models

In [ ]:
# exports
from dataclasses import dataclass
from typing import Optional

import torch
import torch.nn
import torch.optim
from torch import nn as nn
from torch.nn import functional as F, Module
from torch.utils.data import DataLoader, Dataset

from batchbald_redux.active_learning import RandomFixedLengthSampler
from batchbald_redux.black_box_model_training import train
from batchbald_redux.consistent_mc_dropout import (
    BayesianModule,
    ConsistentMCDropout,
    ConsistentMCDropout2d,
)

from batchbald_redux.model_optimizer_factory import ModelOptimizer, ModelOptimizerFactory

In [ ]:
# exports
from batchbald_redux.trained_model import ModelTrainer, TrainedModel, TrainedBayesianModel


class BayesianMNISTCNN(BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input

In [ ]:
BayesianMNISTCNN()

BayesianMNISTCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_drop): ConsistentMCDropout2d(p=0.5)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): ConsistentMCDropout2d(p=0.5)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc1_drop): ConsistentMCDropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# exports


class BayesianMNISTCNN_EBM(BayesianModule):
    """Without Softmax."""

    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)

        return input

In [ ]:
BayesianMNISTCNN_EBM()

BayesianMNISTCNN_EBM(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_drop): ConsistentMCDropout2d(p=0.5)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): ConsistentMCDropout2d(p=0.5)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc1_drop): ConsistentMCDropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# exports


class MnistOptimizerFactory(ModelOptimizerFactory):
    def create_model_optimizer(self) -> ModelOptimizer:
        model = BayesianMNISTCNN()
        optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-4)
        return ModelOptimizer(model=model, optimizer=optimizer)


@dataclass
class MnistModelTrainer(ModelTrainer):
    device: str

    num_training_samples: int = 1
    num_validation_samples: int = 20
    num_patience_epochs: int = 20
    max_training_epochs: int = 120

    min_samples_per_epoch: int = 1024
    num_training_batch_size: int = 64
    num_evaluation_batch_size: int = 128

    @staticmethod
    def create_model_optimizer() -> ModelOptimizer:
        model = BayesianMNISTCNN()
        optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-4)
        return ModelOptimizer(model=model, optimizer=optimizer)

    def get_train_dataloader(self, dataset: Dataset):
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.num_training_batch_size,
            sampler=RandomFixedLengthSampler(dataset, self.min_samples_per_epoch),
            drop_last=True,
        )
        return train_loader

    def get_evaluation_dataloader(self, dataset: Dataset):
        evaluation_loader = torch.utils.data.DataLoader(
            dataset, batch_size=self.num_evaluation_batch_size, drop_last=False, shuffle=False
        )
        return evaluation_loader

    def get_trained(self, *, train_loader: DataLoader, train_augmentations: Optional[Module], validation_loader: DataLoader,
                    log) -> TrainedModel:
        model_optimizer = self.create_model_optimizer()

        train(
            model=model_optimizer.model,
            optimizer=model_optimizer.optimizer,
            training_samples=self.num_training_samples,
            validation_samples=self.num_validation_samples,
            train_loader=train_loader,
            train_augmentations=train_augmentations,
            validation_loader=validation_loader,
            patience=self.num_patience_epochs,
            max_epochs=self.max_training_epochs,
            device=self.device,
            training_log=log,
        )

        return TrainedBayesianModel(model_optimizer.model)

    def get_distilled(self, *, prediction_loader: DataLoader, train_augmentations: Optional[Module],
                      validation_loader: DataLoader, log) -> TrainedModel:
        model_optimizer = self.create_model_optimizer()

        loss = torch.nn.KLDivLoss(log_target=True, reduction="batchmean")

        train(
            model=model_optimizer.model,
            optimizer=model_optimizer.optimizer,
            loss=loss,
            validation_loss=torch.nn.NLLLoss(),
            training_samples=self.num_training_samples,
            validation_samples=self.num_validation_samples,
            train_loader=prediction_loader,
            train_augmentations=train_augmentations,
            validation_loader=validation_loader,
            patience=self.num_patience_epochs,
            max_epochs=self.max_training_epochs,
            prefer_accuracy=True,
            device=self.device,
            training_log=log,
        )

        return TrainedBayesianModel(model_optimizer.model)

In [ ]:
# slow

from batchbald_redux import dataset_challenges

fast_mnist_train, fast_mnist_test = dataset_challenges.create_MNIST_dataset("cuda")

model_trainer = MnistModelTrainer("cuda")

train_loader = model_trainer.get_train_dataloader(fast_mnist_train)

test_loader = model_trainer.get_evaluation_dataloader(fast_mnist_test)

log = {}
trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=None, validation_loader=test_loader, log=log)

In [ ]:
# slow

subset_mnist = fast_mnist_train * 0.2
print(subset_mnist)

train_loader = model_trainer.get_train_dataloader(subset_mnist)

test_loader = model_trainer.get_evaluation_dataloader(fast_mnist_test)

log = {}
trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=None, validation_loader=test_loader, log=log)

('FastMNIST (Train)')~x0.2


  1%|          | 1/120 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.4094, 'crossentropy': 1.9953440071105957}


...

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.9803, 'crossentropy': 0.14164143002033233}
RestoringEarlyStopping: 3 / 20


[1/16]   6%|6          [00:00<?]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.9807, 'crossentropy': 0.14529887140989303}
RestoringEarlyStopping: 4 / 20
RestoringEarlyStopping: Restoring best parameters. (Score: 0.9848)
RestoringEarlyStopping: Restoring optimizer.
